In [1]:
from utils import PredictorSPlitModel, arr2db, forna_url, DataEncoder

Using TensorFlow backend.


In [2]:
# running on CPU is extremely slow!

In [3]:
import numpy as np

In [4]:
from collections import Counter

In [5]:
import keras

In [6]:
# model = Predictor('model/2019_09_08.1/model.hdf5')
model = PredictorSPlitModel('model/2019_09_12.1/model.hdf5')

4


utils.py:366: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
  output=[layer_final_hidden.output, layer_fe.output])
utils.py:376: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ar..., inputs=[<tf.Tenso...)`
  output=new_output)


In [7]:
# seq='GGCGCAGUGGGCUAGCGCCACUCAAAAGGCCCAU'

# seq='GUUGUGAAAU'

seq = 'GAUCACCUUUGGAUC'

# y, logp, fe = model.predict_one_step_ar(seq=seq, 
#                                        n_sample=100, start_offset=2, p_clip=1e-12)

y, logp, fe = model.predict_one_step_ar(seq=seq, 
                                       n_sample=10, start_offset=2, p_clip=1e-12)


 46%|████▌     | 6/13 [00:00<00:01,  5.57it/s]utils.py:483: RuntimeWarning: divide by zero encountered in log
  _lp = vals_sampled * np.log(_vals) + (1 - vals_sampled) * np.log(1 - _vals)
100%|██████████| 13/13 [00:00<00:00, 24.00it/s]


In [8]:
y.shape

(10, 15, 15, 1)

In [9]:
# idx_example = 0
# idx1 = 3
# idx2 = 6

# idx_example = 0
# idx1 = 4
# idx2 = 13

idx_example = 0
idx1 = 4
idx2 = 10

# idx_example = 0
# idx1 = 5
# idx2 = 9

# make sure to pick the index where the current val is 0 to do GA
assert y[idx_example, idx1, idx2, 0] == 0

In [10]:

# x_ite, seq_new = model.gradient_ascent(seq=seq, y_prev=y[idx_example, :, :, 0], idx1=idx1, idx2=idx2, 
#                                        n_ite=1000, lr=0.1, verbose=True)

x_ite, seq_new = model.gradient_ascent(seq=seq, y_prev=y[idx_example, :, :, 0], idx1=idx1, idx2=idx2, 
                                       n_ite=100, lr=0.1, verbose=True)


0 5.326366e-07
1 6.4237014e-07
2 7.5124166e-07
3 8.826979e-07
4 1.0960549e-06
5 1.6026122e-06
6 2.2043012e-06
7 2.79003e-06
8 3.7805123e-06
9 5.787541e-06
10 4.762825e-05
11 0.2417653
12 0.9408141
13 0.9829748
14 0.98813546
15 0.99117273
16 0.99261624
17 0.9919166
18 0.99309134
19 0.9924992
20 0.99353796
21 0.99321413
22 0.9934662
23 0.99515295
24 0.9946128
25 0.9959888
26 0.99595046
27 0.9956611
28 0.9959726
29 0.99616456
30 0.99606436
31 0.9963852
32 0.9952944
33 0.99637234
34 0.9957436
35 0.9964162
36 0.99582666
37 0.9963349
38 0.9957444
39 0.99617165
40 0.9964629
41 0.99645597
42 0.9964604
43 0.995996
44 0.9969778
45 0.9957314
46 0.99710566
47 0.99718696
48 0.9977144
49 0.99770457
50 0.99795973
51 0.9973303
52 0.9980464
53 0.9976763
54 0.9981383
55 0.99787545
56 0.99826056
57 0.99830806
58 0.99856746
59 0.9984861
60 0.99883455
61 0.99891984
62 0.9990219
63 0.9988638
64 0.99904746
65 0.99838686
66 0.9989392
67 0.9990815
68 0.9987656
69 0.99900705
70 0.9986914
71 0.99904555
72 0.9986

In [11]:
seq

'GAUCACCUUUGGAUC'

In [12]:
seq_new

'GAUCCGCUUGGGAUC'

In [13]:
seq_new == seq

False

In [14]:
# x_ite